This 3A notebook will be perform the second half of NLP preprocessing using spaCy, mostly.  Also, this will produce a large dataframe two lyric sets (with/with out stopwords), each with word counts and spaCy .doc (vectorization, POS tags, Named Entity recognition, etc.).

Pulling down the dataset which completed some, but not all, of the NLP preprocessing in NB 3 (Using the Fake News group's pre-processing notebook as a guide (https://github.com/georgetown-analytics/From-Russia-With-Love-fake-news-/blob/master/Notebooks/Step_1_Data_Cleaning.ipynb)). 

This particular set has been through: lowercase, remove URLs, ww, special characters, extra whitespace, accented characters and expanded contractions.

Will run spaCy and append the nlp.doc, on the (nearly) complete lyrics.  Add word / letter counts to the df, run affinity and sentiment append to df.

Then will remove stopwords. And do spaCy, counts, affinity, sentiment.  

At the end of this NB we will have four possible paths to run EDA or modeling IOT see impacts of various pre-processing choices:
1) Semi-clean, full lyrics.
2) Fully clean, NLTK NLP pre-processing (affinity and sentiment).
3) Semi-clean, full lyrics, spaCy vectorization and NLTK pre-processing.
4) Fully clean, spaCy vectorization and NLTK pre-processing.

In [1]:
import s3fs
import pandas as pd
pd.set_option('display.max_columns', 100000)
pd.set_option('display.max_row', 1000000)
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import re
# import tldextract """Accurately separate the TLD from the registered domain and subdomains of a URL"""
from tqdm.autonotebook import tqdm
tqdm.pandas(desc="progress-bar", leave=False)
import string

import spacy
from spacy.lang import punctuation
from spacy.lang.en import English
from spacy import displacy
nlp = spacy.load("en_core_web_lg")

import unicodedata  # might need to pip install unicodedate2 on aws sagemaker
import contractions
from contractions import contractions_dict ## pip installed this
from textblob import TextBlob
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import STOPWORDS
import warnings
from afinn import Afinn
warnings.filterwarnings('ignore')

%matplotlib inline
sns.set(style='darkgrid',palette='Dark2',rc={'figure.figsize':(9,6),'figure.dpi':90})

punctuation = string.punctuation + '”' + '“' + '–' + '““' + "’’" + '”'
stopword = stopwords.words('english')
stopwords = set(STOPWORDS)
wordnet_lemmatizer = WordNetLemmatizer()

<ipython-input-1-4fce9e1e4886>:10: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [20]:
from datetime import datetime
print("Current Time =", datetime.now())

Current Time = 2021-05-23 09:15:19.340291


In [2]:
fs = s3fs.S3FileSystem(anon=False,key='###'',secret='###')

g_df = pd.read_csv('s3://music-lyrics-chain/genres_midcln_df.csv')

In [3]:
g_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86391 entries, 0 to 86390
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   original_csv  86391 non-null  object 
 1   artist_name   86391 non-null  object 
 2   song_name     86391 non-null  object 
 3   link          86391 non-null  object 
 4   lyrics        86391 non-null  object 
 5   language      86391 non-null  object 
 6   genre         86391 non-null  object 
 7   date          0 non-null      float64
dtypes: float64(1), object(7)
memory usage: 5.3+ MB


In [4]:
g_df.shape

(86391, 8)

In [5]:
g_df.iloc[84304]['lyrics']#genres Taylor Swift The Story of Us

'i used to think one day we would tell the story of us. how we met and sparks flew instantly. people would say they are the lucky ones. i used to know my place was a spot next to you. now I am searching the room for an empty seat. because lately i do not even know what page you are on. oh. a simple complication, miscommunication. has lead to fallout. too many things that i wish you knew. so many walls up i can not break through. now I am standing alone in a crowded room. and we are not speaking. and I am dying to know. is it killing you like it is killing me?. and i do not know what to say. since the twist of fate. when it all broke down. and the story of us looks a lot like a tragedy now. next chapter. how did we end up this way?. se me nervously pulling at my clothes. and trying to look busy. and you are doing your best to avoid me. I am starting to think one day I will tell the story of us. how i was losing my mind when i saw you here. but you held your pride like you should have he

In [6]:
# Two additional cleanups.  Found during further EDA.  Remove tiny genres, remove three
#songs that have NaN lyrics.

g_df.drop(g_df[g_df['genre']=='Samba'].index, inplace = True)
g_df.drop(g_df[g_df['genre']=='Sertanejo'].index, inplace = True)
g_df.drop(g_df[g_df['genre']=='Funk Carioca'].index, inplace = True)

g_df = g_df.dropna(axis=0, subset=['lyrics'])

In [7]:
lyrics = [_ for _ in g_df['lyrics']]

def set_doc(lyrics):
    scores = []
    for t in lyrics:
        doc = nlp(t)
        scores.append(doc)
    return scores

new_doc = set_doc(g_df['lyrics'].tolist())

g_df['spaCy_Doc'] = new_doc

%time

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.1 µs


I don't know what %time is telling me.  This nlp(t) step took over an hour.

In [8]:
g_df.shape

(86294, 9)

In [9]:
g_df.head()

,original_csv,artist_name,song_name,link,lyrics,language,genre,date,spaCy_Doc
0,genres_csv,10000-maniacs,More Than This,/10000-maniacs/more-than-this.html,i could feel at the time. there was no way of ...,ENGLISH,Rock,NaN,"(i, could, feel, at, the, time, ., there, was,..."
1,genres_csv,10000-maniacs,Because The Night,/10000-maniacs/because-the-night.html,"take me now, baby, here as i am. hold me close...",ENGLISH,Rock,NaN,"(take, me, now, ,, baby, ,, here, as, i, am, ...."
2,genres_csv,10000-maniacs,These Are Days,/10000-maniacs/these-are-days.html,these are. these are days you will remember. n...,ENGLISH,Rock,NaN,"(these, are, ., these, are, days, you, will, r..."
3,genres_csv,10000-maniacs,A Campfire Song,/10000-maniacs/a-campfire-song.html,"a lie to say, ""of my mountain has coal veins a...",ENGLISH,Rock,NaN,"(a, lie, to, say, ,, "", of, my, mountain, has,..."
4,genres_csv,10000-maniacs,Everyday Is Like Sunday,/10000-maniacs/everyday-is-like-sunday.html,trudging slowly over wet sand. back to the ben...,ENGLISH,Rock,NaN,"(trudging, slowly, over, wet, sand, ., back, t..."


In [10]:
g_df.iloc[84304]['spaCy_Doc']#genres Taylor Swift The Story of Us

if you had only gotten to me first. if i could write a book on how this hurts. if you could see the way your smile looks to me. if i was not scared of the world alone. if i did not know he loves me so. if i could only find the strength to leave. you would be with me, so baby. do not you look in my eyes. i might fall in love. with a beautiful boy. i wish that i could touch. you go i hope you think of me. as the stars fall down and the heavens fade. this girl is begging wait for me. it is so wrong that all i want to do. is forget it all and just be with you. i have to go because he do not know. but the writings on the wall. but here we are in so deep. soaking wet. jumping in without no safety net. and i do not know what kind of ropes. would break this kind of fall, so baby. do not you look in my eyes. i might fall in love. with a beautiful boy. i wish that i could touch. you go, i hope you think of me. as the stars fall down and the heavens fade. this girl is begging wait, wait for me. w

Affinity Score - Sentiment Analysis.  This uses AFINN Lexicon, which is for microblogs (tweets).  This may not be the best sentiment lexicon for songs.  spaCy uses a different one.  Another reason to have a branch off of this trunk.

In [11]:
afinn = Afinn()

def get_affinity_scores(lyrics):
    scores = []
    count = 0
    for t in lyrics:
        if len(t) > 0:
            scores.append(afinn.score(t) / len(t))
        else:
            count += 1
            scores.append(0)
    return scores

%time

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.05 µs


In [14]:
new_affin = get_affinity_scores(g_df['lyrics'].tolist())
g_df['content_affin'] = new_affin

In [15]:
%%time
# word counts
g_df['word_count'] = g_df["lyrics"].apply(lambda x: len(str(x).split(" ")))

# Character counts
g_df['character_count'] = g_df["lyrics"].apply(lambda x: sum(len(word) for word in str(x).split(" ")))

#average word length
g_df['avg_word_length'] = g_df['character_count'] / g_df['word_count']

CPU times: user 3.14 s, sys: 5.73 ms, total: 3.14 s
Wall time: 3.15 s


In [16]:
g_df.head()

,original_csv,artist_name,song_name,link,lyrics,language,genre,date,spaCy_Doc,content_affin,word_count,character_count,avg_word_length
0,genres_csv,10000-maniacs,More Than This,/10000-maniacs/more-than-this.html,i could feel at the time. there was no way of ...,ENGLISH,Rock,NaN,"(i, could, feel, at, the, time, ., there, was,...",0.006545,154,611,3.967532
1,genres_csv,10000-maniacs,Because The Night,/10000-maniacs/because-the-night.html,"take me now, baby, here as i am. hold me close...",ENGLISH,Rock,NaN,"(take, me, now, ,, baby, ,, here, as, i, am, ....",-0.006309,246,1023,4.158537
2,genres_csv,10000-maniacs,These Are Days,/10000-maniacs/these-are-days.html,these are. these are days you will remember. n...,ENGLISH,Rock,NaN,"(these, are, ., these, are, days, you, will, r...",0.018952,181,717,3.961326
3,genres_csv,10000-maniacs,A Campfire Song,/10000-maniacs/a-campfire-song.html,"a lie to say, ""of my mountain has coal veins a...",ENGLISH,Rock,NaN,"(a, lie, to, say, ,, "", of, my, mountain, has,...",-0.015408,272,1027,3.775735
4,genres_csv,10000-maniacs,Everyday Is Like Sunday,/10000-maniacs/everyday-is-like-sunday.html,trudging slowly over wet sand. back to the ben...,ENGLISH,Rock,NaN,"(trudging, slowly, over, wet, sand, ., back, t...",0.006192,117,530,4.529915


In [19]:
#Save an intermediate df as it took awhile to get.

#!touch ../../g_spaCy3_df.csv
#g_df.to_csv('../../g_spaCy3_df.csv', index= False)
!aws s3 cp ../../g_spaCy3_df.csv s3://music-lyrics-chain/g_spaCy3_df.csv
%time

upload: ../../g_spaCy3_df.csv to s3://music-lyrics-chain/g_spaCy3_df.csv
CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 10 µs


Adding the NLTK vector to this, to keep it aligned with the FNW, and have all data available in one dataset.

In [23]:
print("Current Time =", datetime.now())

Current Time = 2021-05-23 09:17:27.340695


In [24]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
  

def lemmatized_word(text):

    word_tokens = nltk.word_tokenize(text)
    lemmatized_word = [wordnet_lemmatizer.lemmatize(word) for word in word_tokens]
    return  " ".join(lemmatized_word) #combine the words into a giant string that vectorizer can accept

g_df['vector'] = g_df['lyrics'].apply(lemmatized_word)

print("Current Time =", datetime.now())

[nltk_data] Downloading package punkt to /Users/Gretzky/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Gretzky/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Current Time = 2021-05-23 09:22:27.330006


In [25]:
g_df['vector'] = g_df['lyrics'].apply(lemmatized_word)

print("Current Time =", datetime.now())

Current Time = 2021-05-23 09:27:21.955303


In [26]:
g_df.head()

,original_csv,artist_name,song_name,link,lyrics,language,genre,date,spaCy_Doc,content_affin,word_count,character_count,avg_word_length,vector
0,genres_csv,10000-maniacs,More Than This,/10000-maniacs/more-than-this.html,i could feel at the time. there was no way of ...,ENGLISH,Rock,NaN,"(i, could, feel, at, the, time, ., there, was,...",0.006545,154,611,3.967532,i could feel at the time . there wa no way of ...
1,genres_csv,10000-maniacs,Because The Night,/10000-maniacs/because-the-night.html,"take me now, baby, here as i am. hold me close...",ENGLISH,Rock,NaN,"(take, me, now, ,, baby, ,, here, as, i, am, ....",-0.006309,246,1023,4.158537,"take me now , baby , here a i am . hold me clo..."
2,genres_csv,10000-maniacs,These Are Days,/10000-maniacs/these-are-days.html,these are. these are days you will remember. n...,ENGLISH,Rock,NaN,"(these, are, ., these, are, days, you, will, r...",0.018952,181,717,3.961326,these are . these are day you will remember . ...
3,genres_csv,10000-maniacs,A Campfire Song,/10000-maniacs/a-campfire-song.html,"a lie to say, ""of my mountain has coal veins a...",ENGLISH,Rock,NaN,"(a, lie, to, say, ,, "", of, my, mountain, has,...",-0.015408,272,1027,3.775735,"a lie to say , `` of my mountain ha coal vein ..."
4,genres_csv,10000-maniacs,Everyday Is Like Sunday,/10000-maniacs/everyday-is-like-sunday.html,trudging slowly over wet sand. back to the ben...,ENGLISH,Rock,NaN,"(trudging, slowly, over, wet, sand, ., back, t...",0.006192,117,530,4.529915,trudging slowly over wet sand . back to the be...


Identify Sentiment, using affinity score.

In [27]:
print("Current Time =", datetime.now())

Current Time = 2021-05-23 09:30:44.036327


In [28]:
def sentiment_check (text):
    polarity_score = TextBlob(text).sentiment.polarity
    g_df['sent_score'] = polarity_score
    if polarity_score < 0:
        return 'negative'
    elif polarity_score == 0:
        return 'neutral'
    else:
        return 'positive'
    
g_df['sent_label'] = g_df['lyrics'].apply(sentiment_check)

print("Current Time =", datetime.now())

Current Time = 2021-05-23 09:32:30.455851


In [29]:
g_df.head()

,original_csv,artist_name,song_name,link,lyrics,language,genre,date,spaCy_Doc,content_affin,word_count,character_count,avg_word_length,vector,sent_score,sent_label
0,genres_csv,10000-maniacs,More Than This,/10000-maniacs/more-than-this.html,i could feel at the time. there was no way of ...,ENGLISH,Rock,NaN,"(i, could, feel, at, the, time, ., there, was,...",0.006545,154,611,3.967532,i could feel at the time . there wa no way of ...,0.215404,positive
1,genres_csv,10000-maniacs,Because The Night,/10000-maniacs/because-the-night.html,"take me now, baby, here as i am. hold me close...",ENGLISH,Rock,NaN,"(take, me, now, ,, baby, ,, here, as, i, am, ....",-0.006309,246,1023,4.158537,"take me now , baby , here a i am . hold me clo...",0.215404,positive
2,genres_csv,10000-maniacs,These Are Days,/10000-maniacs/these-are-days.html,these are. these are days you will remember. n...,ENGLISH,Rock,NaN,"(these, are, ., these, are, days, you, will, r...",0.018952,181,717,3.961326,these are . these are day you will remember . ...,0.215404,positive
3,genres_csv,10000-maniacs,A Campfire Song,/10000-maniacs/a-campfire-song.html,"a lie to say, ""of my mountain has coal veins a...",ENGLISH,Rock,NaN,"(a, lie, to, say, ,, "", of, my, mountain, has,...",-0.015408,272,1027,3.775735,"a lie to say , `` of my mountain ha coal vein ...",0.215404,negative
4,genres_csv,10000-maniacs,Everyday Is Like Sunday,/10000-maniacs/everyday-is-like-sunday.html,trudging slowly over wet sand. back to the ben...,ENGLISH,Rock,NaN,"(trudging, slowly, over, wet, sand, ., back, t...",0.006192,117,530,4.529915,trudging slowly over wet sand . back to the be...,0.215404,negative


In [31]:
g_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86294 entries, 0 to 86310
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   original_csv     86294 non-null  object 
 1   artist_name      86294 non-null  object 
 2   song_name        86294 non-null  object 
 3   link             86294 non-null  object 
 4   lyrics           86294 non-null  object 
 5   language         86294 non-null  object 
 6   genre            86294 non-null  object 
 7   date             0 non-null      float64
 8   spaCy_Doc        86294 non-null  object 
 9   content_affin    86294 non-null  float64
 10  word_count       86294 non-null  int64  
 11  character_count  86294 non-null  int64  
 12  avg_word_length  86294 non-null  float64
 13  vector           86294 non-null  object 
 14  sent_score       86294 non-null  float64
 15  sent_label       86294 non-null  object 
dtypes: float64(4), int64(2), object(10)
memory usage: 11.2+ MB

Creating the _sml version of the lyrics, the spaCy_Doc column, affinity, the word counts, sentiment scores.

In [32]:
print("Current Time =", datetime.now())

Current Time = 2021-05-23 09:41:50.749476


In [33]:
def replace_punctuation(text):
    filters = string.punctuation + '”' + '“' + '–' 
    translate_dict = dict((c, " ") for c in filters)   
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)
    return text

def stops_letters(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 2 and token not in stopword:
            result.append(token)
            
    return " ".join(result)

g_df['lyrics_sml'] =g_df['lyrics'].apply(replace_punctuation)
g_df['lyrics_sml'] =g_df['lyrics'].apply(stops_letters)

print("Current Time =", datetime.now())

Current Time = 2021-05-23 09:42:35.519173


In [34]:
g_df.head()

,original_csv,artist_name,song_name,link,lyrics,language,genre,date,spaCy_Doc,content_affin,word_count,character_count,avg_word_length,vector,sent_score,sent_label,lyrics_sml
0,genres_csv,10000-maniacs,More Than This,/10000-maniacs/more-than-this.html,i could feel at the time. there was no way of ...,ENGLISH,Rock,NaN,"(i, could, feel, at, the, time, ., there, was,...",0.006545,154,611,3.967532,i could feel at the time . there wa no way of ...,0.215404,positive,feel time way knowing fallen leaves night blow...
1,genres_csv,10000-maniacs,Because The Night,/10000-maniacs/because-the-night.html,"take me now, baby, here as i am. hold me close...",ENGLISH,Rock,NaN,"(take, me, now, ,, baby, ,, here, as, i, am, ....",-0.006309,246,1023,4.158537,"take me now , baby , here a i am . hold me clo...",0.215404,positive,baby hold close try understand desire hunger b...
2,genres_csv,10000-maniacs,These Are Days,/10000-maniacs/these-are-days.html,these are. these are days you will remember. n...,ENGLISH,Rock,NaN,"(these, are, ., these, are, days, you, will, r...",0.018952,181,717,3.961326,these are . these are day you will remember . ...,0.215404,positive,days remember promise world warm feel know tru...
3,genres_csv,10000-maniacs,A Campfire Song,/10000-maniacs/a-campfire-song.html,"a lie to say, ""of my mountain has coal veins a...",ENGLISH,Rock,NaN,"(a, lie, to, say, ,, "", of, my, mountain, has,...",-0.015408,272,1027,3.775735,"a lie to say , `` of my mountain ha coal vein ...",0.215404,negative,lie mountain coal veins beds dig men axes dig ...
4,genres_csv,10000-maniacs,Everyday Is Like Sunday,/10000-maniacs/everyday-is-like-sunday.html,trudging slowly over wet sand. back to the ben...,ENGLISH,Rock,NaN,"(trudging, slowly, over, wet, sand, ., back, t...",0.006192,117,530,4.529915,trudging slowly over wet sand . back to the be...,0.215404,negative,trudging slowly wet sand bench clothes stolen ...


In [35]:
g_df.iloc[84304]['lyrics_sml']#genres Taylor Swift The Story of Us

'gotten write book hurts way smile looks scared world know loves strength leave baby look eyes fall love beautiful boy wish touch hope think stars fall heavens fade girl begging wait wrong want forget know writings wall deep soaking wet jumping safety net know kind ropes break kind fall baby look eyes fall love beautiful boy wish touch hope think stars fall heavens fade girl begging wait wait right guess use trying look right tell dying hold wrong look eyes fall love beautiful boy wish touch hope think stars fall heavens fade girl begging wait look look fall love look'

In [36]:
print("Current Time =", datetime.now())

Current Time = 2021-05-23 09:49:28.091281


In [37]:
#Save an intermediate df as it took awhile to get.

!touch ../../g_spaCy4_df.csv
g_df.to_csv('../../g_spaCy4_df.csv', index= False)
!aws s3 cp ../../g_spaCy4_df.csv s3://music-lyrics-chain/g_spaCy4_df.csv

print("Current Time =", datetime.now())

upload: ../../g_spaCy4_df.csv to s3://music-lyrics-chain/g_spaCy4_df.csv
Current Time = 2021-05-23 09:51:43.948279


In [38]:
print("Current Time =", datetime.now())

Current Time = 2021-05-23 09:54:08.965216


In [39]:
lyrics = [_ for _ in g_df['lyrics_sml']]

def set_doc(lyrics):
    scores = []
    for t in lyrics:
        doc = nlp(t)
        scores.append(doc)
    return scores

new_doc = set_doc(g_df['lyrics_sml'].tolist())

g_df['spaCy_Doc_sml'] = new_doc

print("Current Time =", datetime.now())

Current Time = 2021-05-23 10:18:34.652399


In [40]:
g_df.head()

,original_csv,artist_name,song_name,link,lyrics,language,genre,date,spaCy_Doc,content_affin,word_count,character_count,avg_word_length,vector,sent_score,sent_label,lyrics_sml,spaCy_Doc_sml
0,genres_csv,10000-maniacs,More Than This,/10000-maniacs/more-than-this.html,i could feel at the time. there was no way of ...,ENGLISH,Rock,NaN,"(i, could, feel, at, the, time, ., there, was,...",0.006545,154,611,3.967532,i could feel at the time . there wa no way of ...,0.215404,positive,feel time way knowing fallen leaves night blow...,"(feel, time, way, knowing, fallen, leaves, nig..."
1,genres_csv,10000-maniacs,Because The Night,/10000-maniacs/because-the-night.html,"take me now, baby, here as i am. hold me close...",ENGLISH,Rock,NaN,"(take, me, now, ,, baby, ,, here, as, i, am, ....",-0.006309,246,1023,4.158537,"take me now , baby , here a i am . hold me clo...",0.215404,positive,baby hold close try understand desire hunger b...,"(baby, hold, close, try, understand, desire, h..."
2,genres_csv,10000-maniacs,These Are Days,/10000-maniacs/these-are-days.html,these are. these are days you will remember. n...,ENGLISH,Rock,NaN,"(these, are, ., these, are, days, you, will, r...",0.018952,181,717,3.961326,these are . these are day you will remember . ...,0.215404,positive,days remember promise world warm feel know tru...,"(days, remember, promise, world, warm, feel, k..."
3,genres_csv,10000-maniacs,A Campfire Song,/10000-maniacs/a-campfire-song.html,"a lie to say, ""of my mountain has coal veins a...",ENGLISH,Rock,NaN,"(a, lie, to, say, ,, "", of, my, mountain, has,...",-0.015408,272,1027,3.775735,"a lie to say , `` of my mountain ha coal vein ...",0.215404,negative,lie mountain coal veins beds dig men axes dig ...,"(lie, mountain, coal, veins, beds, dig, men, a..."
4,genres_csv,10000-maniacs,Everyday Is Like Sunday,/10000-maniacs/everyday-is-like-sunday.html,trudging slowly over wet sand. back to the ben...,ENGLISH,Rock,NaN,"(trudging, slowly, over, wet, sand, ., back, t...",0.006192,117,530,4.529915,trudging slowly over wet sand . back to the be...,0.215404,negative,trudging slowly wet sand bench clothes stolen ...,"(trudging, slowly, wet, sand, bench, clothes, ..."


In [45]:
print("Current Time =", datetime.now())

Current Time = 2021-05-23 10:42:35.062536


In [46]:
afinn = Afinn()

def get_affinity_scores(lyrics):
    scores = []
    count = 0
    for t in lyrics:
        if len(t) > 0:
            scores.append(afinn.score(t) / len(t))
        else:
            count += 1
            scores.append(0)
    return scores

print("Current Time =", datetime.now())

Current Time = 2021-05-23 10:42:38.219289


In [47]:
new_affin = get_affinity_scores(g_df['lyrics_sml'].tolist())
g_df['content_affin_sml'] = new_affin

print("Current Time =", datetime.now())

Current Time = 2021-05-23 10:46:54.145840


In [48]:
g_df.head()

,original_csv,artist_name,song_name,link,lyrics,language,genre,date,spaCy_Doc,content_affin,word_count,character_count,avg_word_length,vector,sent_score,sent_label,lyrics_sml,spaCy_Doc_sml,content_affin_sml
0,genres_csv,10000-maniacs,More Than This,/10000-maniacs/more-than-this.html,i could feel at the time. there was no way of ...,ENGLISH,Rock,NaN,"(i, could, feel, at, the, time, ., there, was,...",0.006545,154,611,3.967532,i could feel at the time . there wa no way of ...,0.215404,positive,feel time way knowing fallen leaves night blow...,"(feel, time, way, knowing, fallen, leaves, nig...",0.041152
1,genres_csv,10000-maniacs,Because The Night,/10000-maniacs/because-the-night.html,"take me now, baby, here as i am. hold me close...",ENGLISH,Rock,NaN,"(take, me, now, ,, baby, ,, here, as, i, am, ....",-0.006309,246,1023,4.158537,"take me now , baby , here a i am . hold me clo...",0.215404,positive,baby hold close try understand desire hunger b...,"(baby, hold, close, try, understand, desire, h...",-0.006711
2,genres_csv,10000-maniacs,These Are Days,/10000-maniacs/these-are-days.html,these are. these are days you will remember. n...,ENGLISH,Rock,NaN,"(these, are, ., these, are, days, you, will, r...",0.018952,181,717,3.961326,these are . these are day you will remember . ...,0.215404,positive,days remember promise world warm feel know tru...,"(days, remember, promise, world, warm, feel, k...",0.057047
3,genres_csv,10000-maniacs,A Campfire Song,/10000-maniacs/a-campfire-song.html,"a lie to say, ""of my mountain has coal veins a...",ENGLISH,Rock,NaN,"(a, lie, to, say, ,, "", of, my, mountain, has,...",-0.015408,272,1027,3.775735,"a lie to say , `` of my mountain ha coal vein ...",0.215404,negative,lie mountain coal veins beds dig men axes dig ...,"(lie, mountain, coal, veins, beds, dig, men, a...",-0.027473
4,genres_csv,10000-maniacs,Everyday Is Like Sunday,/10000-maniacs/everyday-is-like-sunday.html,trudging slowly over wet sand. back to the ben...,ENGLISH,Rock,NaN,"(trudging, slowly, over, wet, sand, ., back, t...",0.006192,117,530,4.529915,trudging slowly over wet sand . back to the be...,0.215404,negative,trudging slowly wet sand bench clothes stolen ...,"(trudging, slowly, wet, sand, bench, clothes, ...",0.009615


In [49]:
# word counts
g_df['word_count_sml'] = g_df['lyrics_sml'].apply(lambda x: len(str(x).split(" ")))

# Character counts
g_df['character_count_sml'] = g_df['lyrics_sml'].apply(lambda x: sum(len(word) for word in str(x).split(" ")))

#average word length
g_df['avg_word_length_sml'] = g_df['character_count_sml'] / g_df['word_count_sml']

In [50]:
g_df.head()

,original_csv,artist_name,song_name,link,lyrics,language,genre,date,spaCy_Doc,content_affin,word_count,character_count,avg_word_length,vector,sent_score,sent_label,lyrics_sml,spaCy_Doc_sml,content_affin_sml,word_count_sml,character_count_sml,avg_word_length_sml
0,genres_csv,10000-maniacs,More Than This,/10000-maniacs/more-than-this.html,i could feel at the time. there was no way of ...,ENGLISH,Rock,NaN,"(i, could, feel, at, the, time, ., there, was,...",0.006545,154,611,3.967532,i could feel at the time . there wa no way of ...,0.215404,positive,feel time way knowing fallen leaves night blow...,"(feel, time, way, knowing, fallen, leaves, nig...",0.041152,42,202,4.809524
1,genres_csv,10000-maniacs,Because The Night,/10000-maniacs/because-the-night.html,"take me now, baby, here as i am. hold me close...",ENGLISH,Rock,NaN,"(take, me, now, ,, baby, ,, here, as, i, am, ....",-0.006309,246,1023,4.158537,"take me now , baby , here a i am . hold me clo...",0.215404,positive,baby hold close try understand desire hunger b...,"(baby, hold, close, try, understand, desire, h...",-0.006711,93,504,5.419355
2,genres_csv,10000-maniacs,These Are Days,/10000-maniacs/these-are-days.html,these are. these are days you will remember. n...,ENGLISH,Rock,NaN,"(these, are, ., these, are, days, you, will, r...",0.018952,181,717,3.961326,these are . these are day you will remember . ...,0.215404,positive,days remember promise world warm feel know tru...,"(days, remember, promise, world, warm, feel, k...",0.057047,49,250,5.102041
3,genres_csv,10000-maniacs,A Campfire Song,/10000-maniacs/a-campfire-song.html,"a lie to say, ""of my mountain has coal veins a...",ENGLISH,Rock,NaN,"(a, lie, to, say, ,, "", of, my, mountain, has,...",-0.015408,272,1027,3.775735,"a lie to say , `` of my mountain ha coal vein ...",0.215404,negative,lie mountain coal veins beds dig men axes dig ...,"(lie, mountain, coal, veins, beds, dig, men, a...",-0.027473,95,452,4.757895
4,genres_csv,10000-maniacs,Everyday Is Like Sunday,/10000-maniacs/everyday-is-like-sunday.html,trudging slowly over wet sand. back to the ben...,ENGLISH,Rock,NaN,"(trudging, slowly, over, wet, sand, ., back, t...",0.006192,117,530,4.529915,trudging slowly over wet sand . back to the be...,0.215404,negative,trudging slowly wet sand bench clothes stolen ...,"(trudging, slowly, wet, sand, bench, clothes, ...",0.009615,64,353,5.515625


Adding the NLTK vector to this, to keep it aligned with the FNW, and have all data available in one dataset.

Note: This should be exactly the same as the vector column developed in the Step_3_NLP_preprocessing NB.

In [51]:
print("Current Time =", datetime.now())

Current Time = 2021-05-23 10:50:11.209643


In [52]:
g_df['vector_sml'] = g_df['lyrics_sml'].apply(lemmatized_word)

print("Current Time =", datetime.now())

Current Time = 2021-05-23 10:51:08.039842


In [53]:
g_df.head()

,original_csv,artist_name,song_name,link,lyrics,language,genre,date,spaCy_Doc,content_affin,word_count,character_count,avg_word_length,vector,sent_score,sent_label,lyrics_sml,spaCy_Doc_sml,content_affin_sml,word_count_sml,character_count_sml,avg_word_length_sml,vector_sml
0,genres_csv,10000-maniacs,More Than This,/10000-maniacs/more-than-this.html,i could feel at the time. there was no way of ...,ENGLISH,Rock,NaN,"(i, could, feel, at, the, time, ., there, was,...",0.006545,154,611,3.967532,i could feel at the time . there wa no way of ...,0.215404,positive,feel time way knowing fallen leaves night blow...,"(feel, time, way, knowing, fallen, leaves, nig...",0.041152,42,202,4.809524,feel time way knowing fallen leaf night blowin...
1,genres_csv,10000-maniacs,Because The Night,/10000-maniacs/because-the-night.html,"take me now, baby, here as i am. hold me close...",ENGLISH,Rock,NaN,"(take, me, now, ,, baby, ,, here, as, i, am, ....",-0.006309,246,1023,4.158537,"take me now , baby , here a i am . hold me clo...",0.215404,positive,baby hold close try understand desire hunger b...,"(baby, hold, close, try, understand, desire, h...",-0.006711,93,504,5.419355,baby hold close try understand desire hunger b...
2,genres_csv,10000-maniacs,These Are Days,/10000-maniacs/these-are-days.html,these are. these are days you will remember. n...,ENGLISH,Rock,NaN,"(these, are, ., these, are, days, you, will, r...",0.018952,181,717,3.961326,these are . these are day you will remember . ...,0.215404,positive,days remember promise world warm feel know tru...,"(days, remember, promise, world, warm, feel, k...",0.057047,49,250,5.102041,day remember promise world warm feel know true...
3,genres_csv,10000-maniacs,A Campfire Song,/10000-maniacs/a-campfire-song.html,"a lie to say, ""of my mountain has coal veins a...",ENGLISH,Rock,NaN,"(a, lie, to, say, ,, "", of, my, mountain, has,...",-0.015408,272,1027,3.775735,"a lie to say , `` of my mountain ha coal vein ...",0.215404,negative,lie mountain coal veins beds dig men axes dig ...,"(lie, mountain, coal, veins, beds, dig, men, a...",-0.027473,95,452,4.757895,lie mountain coal vein bed dig men ax dig lie ...
4,genres_csv,10000-maniacs,Everyday Is Like Sunday,/10000-maniacs/everyday-is-like-sunday.html,trudging slowly over wet sand. back to the ben...,ENGLISH,Rock,NaN,"(trudging, slowly, over, wet, sand, ., back, t...",0.006192,117,530,4.529915,trudging slowly over wet sand . back to the be...,0.215404,negative,trudging slowly wet sand bench clothes stolen ...,"(trudging, slowly, wet, sand, bench, clothes, ...",0.009615,64,353,5.515625,trudging slowly wet sand bench clothes stolen ...


Identify Sentiment on the smaller set, using affinity score.

In [56]:
def sentiment_check (text):
    polarity_score = TextBlob(text).sentiment.polarity
    g_df['sent_score_sml'] = polarity_score
    if polarity_score < 0:
        return 'negative'
    elif polarity_score == 0:
        return 'neutral'
    else:
        return 'positive'
    
g_df['sent_label_sml'] = g_df['lyrics_sml'].apply(sentiment_check)

In [57]:
g_df.describe(include='all')

,original_csv,artist_name,song_name,link,lyrics,language,genre,date,spaCy_Doc,content_affin,word_count,character_count,avg_word_length,vector,sent_score,sent_label,lyrics_sml,spaCy_Doc_sml,content_affin_sml,word_count_sml,character_count_sml,avg_word_length_sml,vector_sml,sent_score_sml,sent_label_sml
count,86294,86294,86294,86294,86294,86294,86294,0.0,86294,86294.000000,86294.000000,86294.000000,86294.000000,86294,8.629400e+04,86294,86294,86294,86294.000000,86294.000000,86294.000000,86294.000000,86294,8.629400e+04,86294
unique,1,1110,66802,86294,86215,1,3,NaN,86294,NaN,NaN,NaN,NaN,86206,NaN,3,85379,86294,NaN,NaN,NaN,NaN,85356,NaN,3
top,genres_csv,elvis-presley,Intro,/the-maine/free.html,"so, so you think you can tell. heaven from hel...",ENGLISH,Rock,NaN,"(intro, :, jay, z, snippet, ], ., bring, , th...",NaN,NaN,NaN,NaN,"so , so you think you can tell . heaven from h...",NaN,positive,holding rope got feet ground hearing sound tel...,"(things, unnecessary, understand, love, key, s...",NaN,NaN,NaN,NaN,dreaming white christmas like one know treetop...,NaN,positive
freq,86294,747,50,1,3,86294,47408,NaN,1,NaN,NaN,NaN,NaN,3,NaN,59877,4,1,NaN,NaN,NaN,NaN,4,NaN,58208
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002884,289.817797,1133.167532,3.942773,NaN,2.154040e-01,NaN,NaN,NaN,0.010879,108.449127,552.435511,5.120099,NaN,2.944444e-01,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.021585,177.280299,692.122705,0.376230,NaN,2.775574e-17,NaN,NaN,NaN,0.046184,74.646489,379.487244,0.468553,NaN,1.110229e-16,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.369973,1.000000,8.000000,1.740741,NaN,2.154040e-01,NaN,NaN,NaN,-0.493151,1.000000,0.000000,0.000000,NaN,2.944444e-01,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.007819,171.000000,678.000000,3.719101,NaN,2.154040e-01,NaN,NaN,NaN,-0.012953,61.000000,312.000000,4.812500,NaN,2.944444e-01,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002400,245.000000,961.000000,3.903517,NaN,2.154040e-01,NaN,NaN,NaN,0.008448,88.000000,450.000000,5.083333,NaN,2.944444e-01,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.013220,357.000000,1381.000000,4.116105,NaN,2.154040e-01,NaN,NaN,NaN,0.032751,129.000000,661.000000,5.384615,NaN,2.944444e-01,NaN


In [58]:
print("Current Time =", datetime.now())

Current Time = 2021-05-23 11:01:06.528871


In [59]:
!touch ../../g_spaCy5_df.csv
g_df.to_csv('../../g_spaCy5_df.csv', index= False)
!aws s3 cp ../../g_spaCy5_df.csv s3://music-lyrics-chain/g_spaCy5_df.csv
    
print("Current Time =", datetime.now())

upload: ../../g_spaCy5_df.csv to s3://music-lyrics-chain/g_spaCy5_df.csv
Current Time = 2021-05-23 11:04:10.369015


In [60]:
g_df.loc[84304]#genres Taylor Swift The Story of Us

original_csv                                                  genres_csv
artist_name                                                 taylor-swift
song_name                                                The Story Of Us
link                                  /taylor-swift/the-story-of-us.html
lyrics                 i used to think one day we would tell the stor...
language                                                         ENGLISH
genre                                                                Pop
date                                                                 NaN
spaCy_Doc              (i, used, to, think, one, day, we, would, tell...
content_affin                                                  -0.012461
word_count                                                           484
character_count                                                     1764
avg_word_length                                                 3.644628
vector                 i used to think one day we w

In [61]:
g_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86294 entries, 0 to 86310
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   original_csv         86294 non-null  object 
 1   artist_name          86294 non-null  object 
 2   song_name            86294 non-null  object 
 3   link                 86294 non-null  object 
 4   lyrics               86294 non-null  object 
 5   language             86294 non-null  object 
 6   genre                86294 non-null  object 
 7   date                 0 non-null      float64
 8   spaCy_Doc            86294 non-null  object 
 9   content_affin        86294 non-null  float64
 10  word_count           86294 non-null  int64  
 11  character_count      86294 non-null  int64  
 12  avg_word_length      86294 non-null  float64
 13  vector               86294 non-null  object 
 14  sent_score           86294 non-null  float64
 15  sent_label           86294 non-null 